# Assignment segment and cluster Boroughs of Toronto - part 2: adding geo data

## Matthias Reumann

In [1]:
# Install the lxml package in the current Jupyter kernel using conda
#import sys
#!conda install --yes --prefix {sys.prefix} lxml

There seem to be issues in installing the lxml package using conda in this environment. 
Edit: The issue seems to be resolved and the results are shown. When running the code locally in a python script, the code executed perfectly.

## Scrap the wikipedia page

In [2]:
import pandas as pd


url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

tables = pd.read_html(url, header=0)
print(tables[0].columns.values)

df = tables[0]
dfToronto = tables[0]
print(dfToronto.head(10))
print(dfToronto.shape)




['Postcode' 'Borough' 'Neighbourhood']
  Postcode           Borough     Neighbourhood
0      M1A      Not assigned      Not assigned
1      M2A      Not assigned      Not assigned
2      M3A        North York         Parkwoods
3      M4A        North York  Victoria Village
4      M5A  Downtown Toronto      Harbourfront
5      M5A  Downtown Toronto       Regent Park
6      M6A        North York  Lawrence Heights
7      M6A        North York    Lawrence Manor
8      M7A      Queen's Park      Not assigned
9      M8A      Not assigned      Not assigned
(288, 3)


The table of Boroughs and neighbourhoods has 288 rows.

## Drop 'not assigned' fields in 'Borough'


In [3]:
# drop 'not assigned' fields in 'Borough'
# Get names of indexes
index = dfToronto[dfToronto['Borough'] == 'Not assigned'].index
# Delete these row indexes from dataFrame
dfToronto.drop(index, inplace=True)
print(dfToronto.shape)
dfToronto.head(10)


(211, 3)


,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


After dropping the 'not assigned' rows in the Borough column, the table has 211 rows.

## Assign Neighbourhood 'Not assigned' to Borough name

In [4]:
# assign Neighbourhood 'Not assigned' to Borough name

index = dfToronto[dfToronto['Neighbourhood'] == 'Not assigned'].index

for i in index:
    print(i)
    dfToronto.loc[i]['Neighbourhood'] = dfToronto.loc[i]['Borough']

print(dfToronto.head(10))
print(dfToronto.shape)


8
   Postcode           Borough     Neighbourhood
2       M3A        North York         Parkwoods
3       M4A        North York  Victoria Village
4       M5A  Downtown Toronto      Harbourfront
5       M5A  Downtown Toronto       Regent Park
6       M6A        North York  Lawrence Heights
7       M6A        North York    Lawrence Manor
8       M7A      Queen's Park      Queen's Park
10      M9A         Etobicoke  Islington Avenue
11      M1B       Scarborough             Rouge
12      M1B       Scarborough           Malvern
(211, 3)


## Groupby Boroughs keeping the Neighbourhood as lists
Also, to keep a unique post code per neighbourhood, the grouping is done by Borough then Postcode. This differs from part 1 of the assignment.


In [5]:
# groupby Boroughs keeping the Neighbourhood as lists
#dfToronto = dfToronto.groupby(['Borough']).agg({'Postcode': list, 'Neighbourhood': list}).reset_index()
dfToronto = dfToronto.groupby(['Borough', 'Postcode']).agg({'Neighbourhood': list}).reset_index()

print(dfToronto.columns.values)
dfToronto = dfToronto[['Postcode', 'Borough', 'Neighbourhood']]
print(dfToronto.shape)
dfToronto

['Borough' 'Postcode' 'Neighbourhood']
(103, 3)


,Postcode,Borough,Neighbourhood
0,M4N,Central Toronto,[Lawrence Park]
1,M4P,Central Toronto,[Davisville North]
2,M4R,Central Toronto,[North Toronto West]
3,M4S,Central Toronto,[Davisville]
4,M4T,Central Toronto,"[Moore Park, Summerhill East]"
...,...,...,...
98,M6C,York,[Humewood-Cedarvale]
99,M6E,York,[Caledonia-Fairbanks]
100,M6M,York,"[Del Ray, Keelesdale, Mount Dennis, Silverthorn]"
101,M6N,York,"[The Junction North, Runnymede]"


Toronto seems to have 11 Boroughs after grouping them. There are 103 Neighbourhoods.

## Add geo data to the dataframe by using the .csv file.

In [7]:
c=pd.read_csv("Geospatial_Coordinates.csv")

print(c.head())

dfToronto['Latitude'] = 0.0
dfToronto['Longitude'] = 0.0

for index, row in dfToronto.iterrows():
    dfToronto.at[index, 'Latitude'] = c.loc[c['Postal Code'] == row['Postcode'], 'Latitude']
    dfToronto.at[index, 'Longitude'] = c.loc[c['Postal Code'] == row['Postcode'], 'Longitude']


print(dfToronto.columns.values)
print(dfToronto.shape)
dfToronto.head()


  Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476
['Postcode' 'Borough' 'Neighbourhood' 'Latitude' 'Longitude']
(103, 5)


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4N,Central Toronto,[Lawrence Park],43.728020,-79.388790
1,M4P,Central Toronto,[Davisville North],43.712751,-79.390197
2,M4R,Central Toronto,[North Toronto West],43.715383,-79.405678
3,M4S,Central Toronto,[Davisville],43.704324,-79.388790
4,M4T,Central Toronto,"[Moore Park, Summerhill East]",43.689574,-79.383160
